In [ ]:
from transformers import AutoTokenizer, AutoModel
from arabert.preprocess import ArabertPreprocessor
from farasa.segmenter import FarasaSegmenter
import torch
import arabicstopwords.arabicstopwords as ast
import string
from html.parser import HTMLParser
import os
import json

In [ ]:
model_name= 'aubmindlab/bert-base-arabertv02'  
tokenizer =AutoTokenizer.from_pretrained(model_name) 
model=AutoModel.from_pretrained(model_name,output_hidden_states=True)

In [ ]:
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def remove_stop_words(text):
    stop_words = ast.stopwords_list()
    
    return ' '.join(word for word in text.split() if word not in stop_words)
def strip_tags(html):
    html=str(html)
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def text_to_tokens(text):
    return tokenizer.tokenize(text)

def embedding(text):
    text = remove_stop_words(remove_punctuation(text))
    inputs = tokenizer.encode_plus(text,return_tensors='pt')
    outputs = model(inputs['input_ids'],inputs['attention_mask'],inputs['token_type_ids'])
    return string,text,outputs['last_hidden_state'].detach().numpy()[0][0].tolist()
    
# tokens =text_to_tokens(remove_stop_words(text)) 

In [ ]:
#reading json products
dir_path=r'/var/www/html/Salla/New/'
products_data=[]
for path in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, path)):
        try:
            file = open(os.path.join(dir_path, path),'r')
            content = json.load(file)
            key_string=list(content.keys())[0]
            for product in content['SELECT smsp.id as product_id, smsp.name as product_name,smsc.id as category_id ,\r\nsmsc.name as category_name,smsp.description as product_description, smsb.name as brand_name\r\nFROM salla_reports.stg_mysql_salla__products smsp \r\nJOIN stg_mysql_salla__product_categories smspc  on stg_mysql_salla__product_categories.product_id = smsp.id \r\nJOIN stg_mysql_salla__categories smsc on stg_mysql_salla__product_categories.category_id = stg_mysql_salla__categories.id \r\nJOIN stg_mysql_salla__brands smsb on smsb.id = smsp.brand_id ']:
                data=product
                data['product_description']=strip_tags(data['product_description'])
                data['product_description_vector'] = embedding(data['product_description'])
                data['product_name_vector'] = embedding(data['name'])
                product_id=product['product_id']
                products_data.append({"index": {"_index": "products_semantic", "_id": product_id}})
                products_data.append(data)
                if len(products_data)>=1000:
                    resp = client.bulk(body=products_data)
                    products_data=[]
                    print("1000 Indexed")
                
        except:
            print("Error Parsing")

resp = client.bulk(body=products_data)

In [123]:
# text1 = "ولن نبالغ إذا قلنا: إن 'هاتف' أو 'كمبيوتر المكتب' في زمننا هذا ضروري"
# text2 = "إن 'هاتف' أو 'كمبيوتر المكتب' في زمننا هذا ضروري"

text1 = "ايفون 17"
text2 = "كفر جوالات"

In [124]:
# embedding(text)

In [125]:
# import required libraries
import numpy as np
from numpy.linalg import norm
 
# define two lists or array
nan, nan, A = embedding(text1)
nan, nan, B  = embedding(text2)


# compute cosine similarity
cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

Cosine Similarity: 0.82694749424007
